In [1]:
import numpy as np

In [2]:
class QuasiMonteCarloOption:
    def __init__(self, dim, m, class_seq):
            self.dim = dim
            self.m = m
            self.class_seq = class_seq
        
    def box_muller(self, U1, U2):
        # преобразование U1 и U2 в массивы NumPy, если они не являются таковыми
        U1 = np.asarray(U1)
        U2 = np.asarray(U2)
        
        R = np.sqrt(-2 * np.log(U1))
        Theta = 2 * np.pi * U2
        
        arr_Z = R * np.cos(Theta)  # генерация нормально распределенных значений
        return arr_Z
        
    def european_option(self, name, T, t, K, r, sigma, S_0, option_type):
        '''
        Вычисление премии европейского опциона (колл или пут).
        option_type: "call" для колл-опциона, "put" для пут-опциона.
        '''
        T = T - t
        sobol = self.class_seq.result(name)
        delta_t = T / (self.dim - 1)
        n = int(len(sobol) / 2)
    
        U1 = sobol[:n]
        U2 = sobol[n:2 * n]
        arr_Z = self.box_muller(U1, U2)
    
        mean_C = 0
        for i in range(n):
            S_i = S_0
            for j in range(self.dim - 1):
                S_i = S_i * np.exp((r - 0.5 * sigma**2) * delta_t + sigma * np.sqrt(delta_t) * arr_Z[i][j])
            if option_type == "call":
                C_i = np.exp(-r * T) * max(S_i - K, 0)
            elif option_type == "put":
                C_i = np.exp(-r * T) * max(K - S_i, 0)
            mean_C += C_i
    
        return mean_C / n



    def asian_option(self, name, T, t,  K, r, sigma, S_0, option_type):
        '''
        Вычисление премии азиатского опциона (колл или пут).
        option_type: "call" для колл-опциона, "put" для пут-опциона.
        '''
        T = T - t
        sobol = self.class_seq.result(name)
        delta_t = T / (self.dim - 1)
        n = int(len(sobol) / 2)
    
        U1 = sobol[:n]
        U2 = sobol[n:2 * n]
        arr_Z = self.box_muller(U1, U2)
    
        arr_C = []
        for i in range(n):
            arr_S_i = []
            for j in range(self.dim - 1):
                if j == 0:
                    S_i = S_0 * np.exp((r - 0.5 * sigma**2) * delta_t + sigma * np.sqrt(delta_t) * arr_Z[i][j])
                else:
                    S_i = arr_S_i[j - 1] * np.exp((r - 0.5 * sigma**2) * delta_t + sigma * np.sqrt(delta_t) * arr_Z[i][j])
                arr_S_i.append(S_i)
    
            S_mean = np.mean(arr_S_i)
            if option_type == "call":
                C_i = np.exp(-r * T) * max(S_mean - K, 0)
            elif option_type == "put":
                C_i = np.exp(-r * T) * max(K - S_mean, 0)
            arr_C.append(C_i)
    
        return np.mean(arr_C)


    
    def hitting_probability(self, S, S_0, B, r, sigma, tau):
    
        k = (math.log(S / S_0) - (r - 0.5 * sigma**2) * tau) / sigma
        m = (math.log(B / S_0) - (r - 0.5 * sigma**2) * tau) / sigma
        prob = 1 - np.exp(2*m*(k-m)/tau)
        return prob

        
    def up_and_out_option(self, name, T, t, K, r, sigma, S_0, B, option_type):
        '''
        Вычисление премии барьерного up-and-out опциона (колл или пут).
        option_type: "call" для колл-опциона, "put" для пут-опциона.
        '''
        # T = T - t
        tau = T - t
        delta_t = tau / (self.dim - 1)
        sobol = self.class_seq.result(name)
        n = int(len(sobol) / 2)
    
        U1 = sobol[:n]
        U2 = sobol[n:2 * n]
        arr_Z = self.box_muller(U1, U2)
    
        total_payoff = 0
        for i in range(n):
            breached = False
            S_i = S_0
    
            for j in range(self.dim - 1):
                S_i = S_i * np.exp((r - 0.5 * sigma**2) * delta_t + sigma * np.sqrt(delta_t) * arr_Z[i][j])
                if S_i > B:
                    breached = True
                    break
    
            if not breached:
                if option_type == "call":
                    C_i = np.exp(-r * tau) * max(S_i - K, 0)
                elif option_type == "put":
                    C_i = np.exp(-r * tau) * max(K - S_i, 0)
            else:
                C_i = 0
    
            total_payoff += C_i
    
        return total_payoff / n

        
    def lookback_option(self, name, T, t, r, sigma, x, y, option_type):
        '''
        Вычисление средней стоимости lookback опциона (колл или пут).
        option_type: "call" для колл-опциона, "put" для пут-опциона.
        '''
        tau = T - t
        t_values = np.linspace(0, tau, self.dim + 1)
        payoff_sum = 0
    
        sobol = self.class_seq.result(name)
        n = int(len(sobol) / 2)
        U1 = sobol[:n]
        U2 = sobol[n:2 * n]
        arr_Z = self.box_muller(U1, U2)
    
        for i in range(n):
            S_i = x
            max_S_i = y
            min_S_i = y
    
            for j in range(self.dim):
                delta_t = t_values[j + 1] - t_values[j]
                S_i = S_i * np.exp((r - 0.5 * sigma**2) * delta_t + sigma * np.sqrt(delta_t) * arr_Z[i][j])
                max_S_i = max(max_S_i, S_i)
                min_S_i = min(min_S_i, S_i)
    
            if option_type == "call":
                C_i = np.exp(-r * tau) * (max_S_i - S_i)
            elif option_type == "put":
                C_i = np.exp(-r * tau) * (S_i - min_S_i)
            payoff_sum += C_i
    
        return payoff_sum / n
